In [130]:
import os
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy
import sys

0.21.1


In [131]:
# print(os.path.abspath(os.curdir))
# os.chdir(os.path.abspath(os.curdir))

In [132]:
clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa    = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version pre-0.18 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version pre-0.18 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version pre-0.18 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator FeatureUnion from version pre-0.18 when using

In [133]:
pred_alc= clf_alc.predict(df)
print(pred_alc[:10])

[0 1 1 1 1 1 1 1 0 1]


In [134]:
from sklearn.metrics import roc_curve, auc

In [135]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import twokenize.twokenize as tokenizer

from pipelines.helpers import ItemGetter

def make_classifier():
    clf = Pipeline([
        ("getter", ItemGetter("text")),
        ("tfidf", TfidfVectorizer()),
        ("clf", LogisticRegression())])

    clf_params = {
        'clf__C': 200,
        'clf__dual': False,
        'clf__max_iter': 100,
        'clf__multi_class': 'ovr',
        'clf__penalty': 'l2',
        'tfidf__tokenizer':tokenizer.tokenize,
        'tfidf__ngram_range':(1, 3),
        'tfidf__max_features':200000
    }

    clf.set_params(**clf_params)
    return clf

In [136]:
alc_labels = []
# extracting pre-labeled classification
for i in range(0,len(df.labels)):
    alc_labels.append(df.labels[i][12])
alc_labels = list(map(int, alc_labels))
print(alc_labels[:10])
print(len(alc_labels))

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
15651


In [137]:
# print(df.text[:2])
# print(alc_labels[:2])

In [138]:
# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, alc_labels, test_size=0.2, random_state=26)
# print(df.labels[:5])

In [139]:
# df[:5]
# X_train[:5]
# X_test[:5]
# y_train[:5]
# y_test[:5]
# print('training set:', len(X_train))
# print('testing set:', len(X_test))

In [140]:
%%time
clf_alc = make_classifier()
clf_alc.fit(X_train, y_train)
# pred_alc_new= clf_alc.predict(X_test)
# pred_alc_new= clf_alc.predict(df)

C:\Users\eddie\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 3.44 s


In [141]:
pred_alc_new= clf_alc.predict(X_test)
# pred_alc_new= clf_alc.predict(df)
print(pred_alc_new[:20])
print(y_test[:20])
# print(len(pred_alc_new))
# print(len(y_test))

[1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 0 0 1 1]
[1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]


In [142]:
# comps = y_test[:100] != pred_alc_new[:100]
comps = y_test != pred_alc_new
# print(comps[:100])
print('length of total comps: ', len(comps))
diff = list(numpy.where(comps)[0])
# print('length of difference: ', len(diff))
# print(diff)
print('pretrained alcohol classifer:  %d different item in whole dataset'%(len(diff)))
acc = (len(comps)-len(diff))/len(comps)*100
print('accuracy: %f%%'%(acc))

length of total comps:  3131
pretrained alcohol classifer:  603 different item in whole dataset
accuracy: 80.740977%


In [143]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
auc_score2 = roc_auc_score(y_test,pred_alc_new)
print('alcohol LR AUC: ', auc_score2)

alcohol LR AUC:  0.7500629999367775


In [144]:
clf_alc_initial.fit(X_train, y_train)

AttributeError: 'Pipeline' object has no attribute 'memory'